In [1]:
import pandas as pd



In [10]:
form_resp = pd.read_csv('../participant data/Playlist Feedback Form (Responses) - Form Responses 1.csv')
new_cols = [
    'ts',
    'email',
    'playlist_url',
    'num_of_songs',
    'satisfaction_1',
    'duration_listened',
    'satisfaction_2',
    'satisfaction_3',
    'matched_taste',
    'novelty',
    'songs_similar',
    'songs_diverse',
    'feedbackl_1',
    'feedback_2'
]
rename_dict = dict(zip(list(form_resp.columns), new_cols))
form_resp.rename(columns=rename_dict, inplace=True)
form_resp

,ts,email,playlist_url,num_of_songs,satisfaction_1,duration_listened,satisfaction_2,satisfaction_3,matched_taste,novelty,songs_similar,songs_diverse,feedbackl_1,feedback_2
0,12/4/2022 14:56:58,aron@kormo.si,https://open.spotify.com/playlist/20jmnaGjYIpn...,10,4,3,4,4,4,4,2,3,NaN,NaN
1,12/4/2022 14:58:57,aron@kormo.si,https://open.spotify.com/playlist/3qchXrMpadh0...,7,2,3,2,2,2,5,4,3,NaN,NaN
2,12/4/2022 15:44:31,bocsardi.marci@gmail.com,https://open.spotify.com/playlist/0vFtD1onguf0...,10,4,2,4,2,5,4,3,4,NaN,NaN
3,12/4/2022 16:10:57,aggariyo@gmail.com,https://open.spotify.com/playlist/645ZDyVdI9vb...,10,4,4,3,4,4,2,4,2,NaN,NaN
4,12/4/2022 16:13:01,aggariyo@gmail.com,https://open.spotify.com/playlist/3oNifxqMCzS0...,10,3,3,2,2,3,4,2,4,NaN,NaN
5,12/5/2022 11:18:05,niels12253@gmail.com,https://open.spotify.com/playlist/2EaJgGx04D8c...,7,3,2,4,3,4,4,1,5,NaN,NaN
6,12/5/2022 11:19:10,niels12253@gmail.com,https://open.spotify.com/playlist/653GcpEzFy5r...,10,2,1,3,3,2,4,1,5,NaN,NaN
